## Fake News Detection with Python

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

Importing Data sets

In [38]:
fake_news = pd.read_csv('Fake.csv')
true_news = pd.read_csv('True.csv')

In [39]:
#fake_news data 
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [40]:
#true_news data 
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [41]:
true_news.shape,fake_news.shape

((21417, 4), (23481, 4))

In [42]:
fake_news['class']=0
true_news['class']=1

In [43]:
# for manual testing
manual_testing_fake = fake_news.tail(10)
manual_testing_true = true_news.tail(10)

In [44]:
# Removing data
for i in range(23470,23481,1):
    fake_news.drop([i],axis=0,inplace=True)

for i in range(23406,23417,1):
    fake_news.drop([i],axis=0,inplace=True)
    

In [45]:
manual_testing = pd.concat([manual_testing_true,manual_testing_fake],axis=0)
manual_testing.to_csv("manual_testing.csv")

In [46]:
merged_data = pd.concat([true_news,fake_news],axis=0)
merged_data.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [47]:
required_df = merged_data.drop(["title","subject","date"],axis=1)
required_df.head()

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [48]:
required_df = required_df.sample(frac=1)
required_df.head()

,text,class
16843,"BAGHDAD/ERBIL, Iraq (Reuters) - Iraqi forces o...",1
11337,BUCHAREST (Reuters) - Romania s ruling Social ...,1
12428,MOSCOW (Reuters) - Russia is ready to consider...,1
11582,ADDIS ABABA (Reuters) - South Sudan s governme...,1
6453,NEW YORK (Reuters) - Senate Republican leader ...,1


In [49]:
required_df.isnull().sum()

text     0
class    0
dtype: int64

### Data Cleaning

In [50]:
def remove_un(data):
    data = data.lower()
    data = re.sub('\[.*?\]','',data)
    data = re.sub('\\W',' ',data)
    data = re.sub('https?://\S+|www.\S+','',data)
    data = re.sub('<.*?>+','',data)
    data = re.sub('[%s]'%re.escape(string.punctuation),'',data)
    data = re.sub('\n','',data)
    data = re.sub('\w*\d\w','',data)
    return data

In [51]:
required_df['text']= required_df['text'].apply(remove_un)

In [52]:
required_df.head()

,text,class
16843,baghdad erbil iraq reuters iraqi forces o...,1
11337,bucharest reuters romania s ruling social ...,1
12428,moscow reuters russia is ready to consider...,1
11582,addis ababa reuters south sudan s governme...,1
6453,new york reuters senate republican leader ...,1


### Vectorization

In [56]:
# dependent and independent variables for calculation
x = required_df['text']
y = required_df['class']

In [58]:
x_train, x_test, y_train,y_test= train_test_split(x,y,test_size=0.25)

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
vectorization = TfidfVectorizer()
xv_train  = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### Logistic Regression

In [64]:
from sklearn.linear_model import LogisticRegression

In [67]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [68]:
LR.score(xv_test,y_test)

0.9882342454764239

In [69]:
pred_LR = LR.predict(xv_test)


In [70]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5912
           1       0.99      0.99      0.99      5307

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219



### Decision Tree Classification

In [71]:
from sklearn.tree import DecisionTreeClassifier

In [73]:
DT  = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [74]:
DT.score(xv_test,y_test)

0.9960780818254746

In [75]:
pred_DT = DT.predict(xv_test)

In [78]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5912
           1       1.00      1.00      1.00      5307

    accuracy                           1.00     11219
   macro avg       1.00      1.00      1.00     11219
weighted avg       1.00      1.00      1.00     11219



### Gradient Boosting Classifier

In [79]:
from sklearn.ensemble import GradientBoostingClassifier

In [80]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [81]:
GBC.score(xv_test,y_test)

0.9946519297620109

In [84]:
pred_GBC = GBC.predict(xv_test)

In [85]:
print(classification_report(y_test,pred_GBC))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5912
           1       0.99      1.00      0.99      5307

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219



### Random Forest Classifier

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [88]:
RFC.score(xv_test,y_test)

0.990640877083519

In [89]:
pred_RFC = RFC.predict(xv_test)

In [90]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5912
           1       0.99      0.99      0.99      5307

    accuracy                           0.99     11219
   macro avg       0.99      0.99      0.99     11219
weighted avg       0.99      0.99      0.99     11219



### Manual Testing

In [106]:
def output_label(n):
    if n ==0:
        return "Fake News"
    elif n==1:
        return "True News"
    else:
        return "ERROR!!"
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(remove_un)
    new_x_test = new_def_test['text']
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    
    return print(f'''\n
                    LR Prediction: {output_label(pred_LR)}\n
                    DT Prediction: {output_label(pred_DT)}\n
                    GBC Prediction: {output_label(pred_GBC)}\n
                    RFC Prediction: {output_label(pred_RFC)}\n
                 ''')

In [107]:
news = input("News Here")
manual_testing(news)

News Here Patrick Henningsen  21st Century WireRemember when the Obama Administration told the world how it hoped to identify 5,000 reliable non-jihadist  moderate  rebels hanging out in Turkey and Jordan, who might want to fight for Washington in Syria? After all the drama over its infamous  train and equip  program to create their own Arab army in Syria, they want to give it another try.This week, Pentagon officials announced their new plan to train up to 7,000 more  moderate  fighters, but this time the project would take place inside Syria (and to hell with international law).We re told that this was requested by Ankara, and with all NATO allies singing the same hymn   claiming that this new effort will help in securing Turkey s porous border with Syria, or so the story goes. Washington s political cover for this is fashioned from the popular post-Paris theme: to protect civilized Europe from invading hordes and the terrorists who hide among them, as stated in the Wall Street Journ



                    LR Prediction: Fake News

                    DT Prediction: Fake News

                    GBC Prediction: Fake News

                    RFC Prediction: Fake News

                 
